# HDAWG

Initialize the `tk.HDAWG` instrument drivers as in the previous example and setup and connect the device. Its device serial is `dev8030` and we connect via ethernet (`1gbe`) to the host at `10.42.0.226`.

In [1]:
import zhinst.toolkit as tk

hdawg = tk.HDAWG("hdawg1", "dev8030", interface="1gbe", host="10.42.0.226")
hdawg.setup()              # set up data server connection
hdawg.connect_device()     # connect device to data server

Successfully connected to data server at 10.42.0.2268004 api version: 6
Successfully connected to device DEV8030 on interface 1GBE


## AWG Cores 

The *HDAWG* features eight *AWG channels* grouped in four *AWG Cores* with two channels each. These four *AWG Cores* can be accessed through the `awgs` property of the instruemnt driver. `awgs` is a list of four HDAWG-specific `AWGCore` objects. The `AWGCore` handles the basic functionality to control the *AWG Core* and program its sequencer to play a specific series of waveforms and commands.  

In [2]:
hdawg.awgs[0]

hdawg1: <zhinst.toolkit.control.drivers.hdawg.AWG object at 0x000001E57692E0B8>
    parent  : <zhinst.toolkit.control.drivers.hdawg.HDAWG object at 0x000001E575340BE0>
    index   : 0
    sequence: 
           type: None
            ('target', <DeviceTypes.HDAWG: 'hdawg'>)
            ('clock_rate', 2400000000.0)
            ('period', 0.0001)
            ('trigger_mode', 'None')
            ('repetitions', 1)
            ('alignment', 'End with Trigger')
            ('n_HW_loop', 1)
            ('dead_time', 5e-06)
            ('trigger_delay', 0)
            ('latency', 1.6e-07)
            ('trigger_cmd_1', '//\n')
            ('trigger_cmd_2', '//\n')
            ('wait_cycles', 0)
            ('dead_cycles', 0)
            ('reset_phase', False)
      IQ Modulation DISABLED

An *AWG Core* features a list of *zhinst-toolkit* `Parameters` as well as methods that allow for a high-level control of the AWG Core. The *Parameters* control the channel outputs as well as their modulation settings.

For instance, the channel outputs can be turned *on* and *off* like this:

In [4]:
hdawg.awgs[0].output1("on")   
hdawg.awgs[0].output2("on")

# are the outputs really on now?
hdawg.awgs[0].outputs()

('on', 'on')

In [5]:
hdawg.awgs[0].outputs(("off", "off"))

## AWG Output Modulation

The output modulation for both channel outputs can be turned on with:

In [10]:
hdawg.awgs[0].enable_iq_modulation()

This method applies the corresponding settings for IQ Modulation using 
one of the internal oscillators and two sine generators. The sines are 
used to modulate the AWG output channels at a 90 degree phase difference. The parameters 
`modulation_freq`, `modulation_phase_shift` and `gain1`, `gain2` 
correspond to the settings of the oscillator and the sine generators.

In [12]:
hdawg.awgs[0].modulation_freq

Node: oscs/0/freq
Description: Sets the modulation frequency of the AWG output channels.
Type: Double
Properties: Read, Write
Unit: Hz
Value: 99999999.99999432

A simple sweep of the modulation frequency could look like this:

In [13]:
import numpy as np
import time

for f in np.linspace(10e6, 20e6, 101):
    hdawg.awgs[0].modulation_freq(f)
    # do something here
    time.sleep(0.1)

The *AWG Core* also holds a `SequenceProgram` that defines the pulse sequence played by the *AWG Sequencer*. The next example will describe how this sequence program is defined and how you can configure it to play the sequence of waveforms that you want to use. 